In [1]:
import json
import pandas as pd
import pymysql
import re
import textwrap
from sqlalchemy import create_engine

#### The sample_data_for_assignment.json file contains 2 keys namely cols and data. Write a program to load this data to MySQL (should be installed locally) table (table_name: json_to_sql_table) under a database

In [2]:
file = 'sample_data_for_assignment.json'
with open(file, encoding='utf-8') as train_file:
    dict_train = json.load(train_file)

In [3]:
def create_DataFrame(data):
    
    df_dict_raw = dict()                             
    for each in data['cols']:
        df_dict_raw[each] = []                       
        
    for each in data['data']:                        
        itr = 0
        for key in df_dict_raw:                      
            df_dict_raw[key].append(each[itr])
            itr += 1
    return pd.DataFrame(df_dict_raw)

In [4]:
new_df = create_DataFrame(dict_train)

In [5]:
engine = create_engine('sqlite://', echo=False) #connecting to an sqlite database

In [6]:
new_df.to_sql('json_to_sql_table', con=engine) #moving it to table created in the sqlite database

In [7]:
df_database = engine.execute("SELECT * FROM json_to_sql_table").fetchall() #retrieving the records from that table

#### Q1. unload the data from MySQL to pandas dataframe

In [8]:
final_df = pd.DataFrame(df_database, columns=['index', 'name', 'phone', 'email', 'address', 'region',
                                             'country', 'list', 'postalZip', 'currency'])

In [9]:
final_df

,index,name,phone,email,address,region,country,list,postalZip,currency
0,0,Winifred Branch,1-251-583-4993,in.mi@protonmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,1,Brielle Davidson,1-632-583-7386,venenatis.lacus@hotmail.ca,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,2,Summer Mcfadden,1-716-949-4663,orci.in@yahoo.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@protonmail.edu,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,4,Barry Conner,(491) 267-4372,lorem@hotmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
...,...,...,...,...,...,...,...,...,...,...
495,495,Adrian Rodriquez,1-425-958-2315,non.quam.pellentesque@hotmail.net,107-4830 Lobortis Avenue,Noord Brabant,Russian Federation,1,49343,$90.17
496,496,Sonia Lamb,1-814-767-3340,suscipit.est@yahoo.com,8062 Lectus St.,Kurgan Oblast,Sweden,9,393741,$56.21
497,497,Kylee Silva,1-196-572-3426,nibh.lacinia@icloud.ca,Ap #336-1000 Diam Street,Drenthe,Vietnam,5,70801,$63.16
498,498,Jeanette Todd,1-384-486-3954,nunc@google.net,Ap #680-5749 Neque Avenue,Chandigarh,Ireland,9,3765 TK,$43.01


#### Q2. write a program to display all data under their respective column name as a pandas dataframe

In [10]:
final_df = final_df.drop('index', axis=1) #dropping the index column as it is not needed
final_df.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@protonmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@hotmail.ca,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@yahoo.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@protonmail.edu,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@hotmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


#### Q3. write a program to change all email address to have the format - abc@gmail.com (note the ending should be gmail.com)

In [11]:
def change_domain_email(mail_domain):
    
    new_domain = []
    
    for domain in mail_domain:
        new_domain.append(''.join([domain.split('@')[0], '@gmail.com']))
    return new_domain

In [12]:
final_df['email'] = change_domain_email(final_df['email'])
final_df.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242-403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,85K 8W1,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


#### Q4. The values for "postalZip" doesn't have same type of data (some are string, some are int, some are alpha numeric). Write a program to convert all values to int. In case of alpha numeric remove the letters and keep only the numbers and convert it to int.

In [13]:
def change_postzip(dig_list):
    new_postzip = []
    for dig in dig_list:
        new_postzip.append(int(''.join(re.findall('\d', str(dig)))))
    return new_postzip

In [14]:
final_df['postalZip'] = change_postzip(final_df['postalZip'])
final_df.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,1-251-583-4993,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242403,$14.59
1,Brielle Davidson,1-632-583-7386,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,1-716-949-4663,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,8581,$36.45
3,Geoffrey Long,(116) 475-4598,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,(491) 267-4372,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


#### Q5. write a separate function which take input data as values under the column name "phone" and processes it to return equivalent ASCII char of the value taken 2 at a time from left to right. Considering the value should be in between 65 to 99. If two-digit number is less than 65 replace it with O (O for Orange)

In [15]:
def encoded_phone_numbers(number_list):
    number_list = change_postzip(number_list)
    encoded_pn = []
    for number in number_list:
        encoded_pn.append(''.join([chr(int(x)) if int(x) > 65 else 'O' for x in textwrap.wrap(str(number),2)]))
    return encoded_pn

In [18]:
encoded_phone_number = encoded_phone_numbers(final_df['phone'])

In [19]:
final_df['phone'] = encoded_phone_number
final_df.head()

,name,phone,email,address,region,country,list,postalZip,currency
0,Winifred Branch,OOOOcO,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242403,$14.59
1,Brielle Davidson,OOOOOO,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,OO^^BO,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,8581,$36.45
3,Geoffrey Long,OOKOb,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,OOCOH,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68


####  At last rename the column "phone" to "coded_phone_number" in the dataframe.


In [20]:
final_df = final_df.rename(columns={'phone':'coded_phone_number'})
final_df.head()

,name,coded_phone_number,email,address,region,country,list,postalZip,currency
0,Winifred Branch,OOOOcO,in.mi@gmail.com,Ap #530-5652 Arcu. Rd.,Rio de Janeiro,Nigeria,19,66242403,$14.59
1,Brielle Davidson,OOOOOO,venenatis.lacus@gmail.com,Ap #630-7765 Molestie Avenue,Aisén,Australia,13,18317,$56.16
2,Summer Mcfadden,OO^^BO,orci.in@gmail.com,"P.O. Box 410, 6638 Sed Avenue",Biobío,Belgium,3,8581,$36.45
3,Geoffrey Long,OOKOb,placerat.eget.venenatis@gmail.com,783-9358 Aliquet Street,North Sulawesi,United States,1,28826,$69.58
4,Barry Conner,OOCOH,lorem@gmail.com,795 Mauris Rd.,Vaupés,China,7,335049,$77.68
